# Working with CNN's for  Computer Vision 

This section covers the convolutional neural networks or covnets widely used in computer vision applications. 


## V02 - Download the dataset and making Train and Test sets

### Download the images from image-net

In [ ]:
import urllib
import cv2
import imutils
import numpy as np
import os

pic_num = 0
dir_name = 'shoe'
wnid = 'n02708093'

neg_images_link = 'http://www.image-net.org/api/text/imagenet.synset.geturls?wnid=' + wnid
neg_image_urls = urllib.request.urlopen(neg_images_link).read().decode()
if not os.path.exists(dir_name):
    os.makedirs(dir_name)

for i in neg_image_urls.split('\n'):
    try:
        print('Downloading ', i)
        urllib.request.urlretrieve(i, dir_name + "/" + str(pic_num) + ".jpg")
        img = cv2.imread(dir_name + "/" + str(pic_num) + ".jpg", cv2.IMREAD_GRAYSCALE)
        resized_image = cv2.resize(img, (1000, 1000))
        cv2.imwrite(dir_name + "/" + str(pic_num) + ".jpg", resized_image)
        pic_num += 1

    except Exception as e:
        print(str(e))

## Loading the dataset, and making Train and Test sets

Here's a script that would load all the downloaded images

In [ ]:
import os
import re
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage, misc
import warnings; warnings.filterwarnings('ignore')

image_size = 80

all_images = []
all_labels = []

# mapping = {0: '/remote', 1:'/scissor'}
mapping = {0: '/shoe', 1:'/clock'}

for k,v in mapping.items():
    for root, dirnames, filenames in os.walk(os.path.abspath('') + v):
        for filename in filenames:
            if re.search("\.(jpg|jpeg|png|bmp|tiff)$", filename):
                filepath = os.path.join(root, filename)
                image = ndimage.imread(filepath, mode="L")
                image_resized = misc.imresize(image, (image_size, image_size))
                all_images.append(image_resized)
                all_labels.append(k)

### Defining a function to shuffle the entire dataset

In [ ]:
def shuffle_batch(X, y):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X)
    batch_idx = list(np.array_split(rnd_idx, 1))
    return X[batch_idx], y[batch_idx]

### Convert the lists to numpy array

In [ ]:
all_images = np.array(all_images)
all_labels = np.array(all_labels)
all_images = np.expand_dims(all_images, axis=3)

print('Images', all_images.shape)
print('Labels', all_labels.shape)

### Shuffle entire dataset

In [ ]:
all_images, all_labels = shuffle_batch(all_images, all_labels)

### Spliting into Train (80%) and Test (20%) sets

In [ ]:
percent = int(len(all_images) * 0.8)

train_images, test_images = all_images[:percent], all_images[percent:]
train_labels, test_labels = all_labels[:percent], all_labels[percent:]

print('Total', len(all_images))
print('Train images', train_images.shape)
print('Test images', test_images.shape)
print('Train labels', train_labels.shape)
print('Test labels', test_labels.shape)

# V03 - Dataset Preprocessing

In [ ]:
test_images = test_images.astype('float32') / 255
train_images = train_images.astype('float32') / 255

In [ ]:
import cv2

def denoise(image):
    image = cv2.GaussianBlur(image, (5, 5), 0)
    return image

def resize(image, size):
    image = cv2.resize(image, (size, size))
    return image

In [ ]:
index = 10
%matplotlib inline
image = train_images[index]
plt.imshow(np.squeeze(image, axis=(2,)))

In [ ]:
image = train_images[index]
image = np.squeeze(image, axis=(2,))

plt.imshow(resize(image, 150))
# plt.imshow(denoise(image))

# V04 - Building CNN Model from scratch

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import models

import tensorflow as tf

print('tf version', tf.__version__)

model = models.Sequential()

model.add(layers.Conv2D(64, (3, 3), activation='relu', 
                        input_shape=(image_size, image_size, 1)))

model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))

model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))

In [ ]:
model.summary()

In [ ]:
model.add(layers.Flatten())
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
model.summary()

### Compile the model with optimizer, loss function and metrics

In [ ]:
model.compile(optimizer='rmsprop',
loss='binary_crossentropy',
metrics=['accuracy'])

### Training the model

In [ ]:
model.fit(train_images, train_labels, epochs=30)

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

In [ ]:
test_loss, test_acc = model.evaluate(train_images, train_labels)

In [ ]:
import time
from random import randint

for i in range(20):
    index = randint(0, len(train_images))
    image = train_images[index]
    label = train_labels[index]
    predicted = 0
    if model.predict([[image]])[0][0] >= 0.5:
        predicted = 1
    print('Actual', label, 'Predicted', predicted, label == predicted)


In [ ]:
index = 60

image = train_images[index]
label = train_labels[index]
plt.imshow(np.squeeze(image, axis=(2,)))

print('Actual', label)

predicted = 0
if model.predict([[image]])[0][0] >= 0.5:
    predicted = 1

print('Predicted', predicted)


In [ ]:
train_images.shape

# Data Augmentation

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot
from keras import backend as K

# datagen = ImageDataGenerator(rotation_range=90)
datagen = ImageDataGenerator(horizontal_flip=True, vertical_flip=True)

datagen.fit(train_images)

rotated_images = []
rotated_images_labels = []

for X_batch, y_batch in datagen.flow(train_images, train_labels, 
                                     batch_size=len(train_images), 
                                     shuffle=False):
    rotated_images = X_batch
    rotated_images_labels = y_batch
    print(len(rotated_images))
    break


In [ ]:
index = 62
%matplotlib inline
image = train_images[index]
image_rotated = rotated_images[index]
plt.imshow(np.squeeze(image, axis=(2,)))

In [ ]:
plt.imshow(np.squeeze(image_rotated, axis=(2,)))